In [ ]:
import pandas as pd
import numpy as np
import re
import os
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel, get_linear_schedule_with_warmup
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, RandomSampler
import random
from transformers import AdamW, AutoTokenizer, AutoModel
from torch.utils.data import Dataset, DataLoader, TensorDataset, RandomSampler
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from torch.nn import Parameter
import math
from torch.optim import Adam
from sklearn.model_selection import KFold
import urllib.request
from typing import List
from functools import partial
from sklearn.model_selection import StratifiedShuffleSplit, GroupKFold, KFold


import torchmetrics
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

In [ ]:
test = pd.read_csv('/kaggle/input/feedback-prize-effectiveness/test.csv')

In [ ]:
test

In [ ]:
seed_num = 22
random.seed(seed_num)
np.random.seed(seed_num)
torch.manual_seed(seed_num)
torch.cuda.manual_seed_all(seed_num)
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

In [ ]:
class FeedbackDataset(Dataset):
    def __init__(self, data, is_train=True):
        super().__init__()
        self.max_length = 512
        self.data = data
        self.tokenizer = AutoTokenizer.from_pretrained('../input/deberta-v3-base/deberta-v3-base')
        self.is_train = is_train
    
    def labeling(self, label):
        new_label = {"Ineffective": 0, "Adequate": 1, "Effective": 2}
        return new_label[label]
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if self.is_train:
            sentence1, sentence2, label = self.data['discourse_text'][idx], self.data['discourse_type'][idx], self.data['discourse_effectiveness'][idx]
            label = self.labeling(label)
            label = torch.LongTensor([label])
        else:
            sentence1, sentence2 = self.data['discourse_text'][idx], self.data['discourse_type'][idx]
        input_ids = self.tokenizer.encode(sentence1, sentence2, truncation=True, max_length=self.max_length, padding='max_length', truncation_strategy='only_first')
        input_ids = torch.LongTensor(input_ids)
        if self.is_train:
            return input_ids, label
        else:
            return input_ids

In [ ]:
class FeedbackModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert_model = AutoModel.from_pretrained('../input/deberta-v3-base/deberta-v3-base')
        self.fc = nn.Linear(self.bert_model.config.hidden_size, 3)
        self.bn = nn.BatchNorm1d(3)
        self._init_params()
        
        
    def _init_params(self):
        nn.init.xavier_normal_(self.fc.weight)
        nn.init.constant_(self.fc.bias, 0)
        nn.init.constant_(self.bn.weight, 1)
        nn.init.constant_(self.bn.bias, 0)
        
    
    def forward(self, input_ids):
        attention_mask = (input_ids!=0).long()
        x = self.bert_model(input_ids, attention_mask=attention_mask)
        x = torch.sum(x.last_hidden_state * attention_mask.unsqueeze(-1), dim=1) / attention_mask.sum(dim=1, keepdims=True)
        output = self.bn(self.fc(x))
        return output

In [ ]:
test_dataset = FeedbackDataset(test, False)

In [ ]:
batch_size = 16
test_dataLoader = DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
folds = 5
pred = []
for i in tqdm(range(folds)) : 
    checkpoint = torch.load(f'/kaggle/input/fb-debertav3base/checkpoint-{i + 1}.pt')
    model = FeedbackModel().to(device)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    result = []
    for input_ids in tqdm(test_dataLoader):
        
        with torch.no_grad():     
            outputs = model(input_ids.to(device))
        result.extend(F.softmax(outputs))    
    pred.append(result)

In [ ]:
output = []
for pred1, pred2, pred3, pred4, pred5 in zip(pred[0], pred[1], pred[2], pred[3], pred[4]):
    output.append(((pred1 + pred2 + pred3 + pred4 + pred5)/5).cpu().numpy())

In [ ]:
result = np.array(output)

In [ ]:
submission = pd.read_csv('/kaggle/input/feedback-prize-effectiveness/sample_submission.csv')

In [ ]:
inef, adw, eff = [], [], []
for r1, r2, r3 in result:
    inef.append(r1)
    adw.append(r2)
    eff.append(r3)

In [ ]:
submission['Ineffective'] = inef
submission['Adequate'] = adw
submission['Effective'] = eff

In [ ]:
submission

In [ ]:
submission.to_csv('submission.csv', index=False)